In [1]:
!pip install BeautifulSoup4

In [2]:
# importing the libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests
import csv

In [3]:
# Step 1: Sending a HTTP request to a URL
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

In [4]:
# Step 2: Parse the html content
soup = BeautifulSoup(html_content, "lxml")

In [5]:
# Step 3: Analyze the HTML tag, where your content lives
postalcode_table = soup.find("table", attrs={"class": "wikitable"})
postalcode_data = postalcode_table.find_all('tr')

In [6]:
# Get headers of table
headers = []
for th in postalcode_data:
    for t in th.find_all('th'):
        headers.append(t.text.replace('\n', ' ').strip())

In [7]:
df = pd.DataFrame(columns = headers)

In [8]:
postcode = 0
borough = 0
neighborhood = 0
for td in postalcode_data:
    i = 0
    for td in td.find_all('td'):
        if i == 0:
            postcode = td.text.strip('\n').replace(']','')
            i = i + 1
        elif i == 1:
            borough = td.text.strip('\n').replace(']','')
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    df = df.append({'PostalCode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

In [9]:
# Ignore cells with a borough that is Not assigned
empty = 'Not assigned'
df = df[df.Borough!=empty]
df = df[df.Borough!= 0]
df.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,df.shape[0]):
    if df.iloc[i][2] == empty:
        df.iloc[i][2] = df.iloc[i][1]
        i = i+1                               
df = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df = df.dropna()
df = df[(df.PostalCode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]
df.head(12)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [10]:
df.shape

(103, 3)